In [ ]:
import Blackjack_Agent as ba
import torch as tc

In [60]:
import numpy as np

In [10]:
def grid_search(lr_list, gamma_list, tau_list, ep_rate):
    results = tc.zeros((len(lr_list)*len(gamma_list)*len(tau_list)*len(ep_rate), 5))
    ind = 0
    
    for i in range(len(lr_list)):
        for j in range(len(gamma_list)):
            for k in range(len(tau_list)):
                for l in range(len(ep_rate)):
                    p = ba.BlackNet(out_mode='tanh')
                    m = ba.Model(p)
                    m.create_memory(12500)
                    m.set_params(128, 25000, lr_list[i], gamma_list[j], tau_list[k], (0.99, 0.05, ep_rate[l]))
                    m.train_agent()
                    ew = 0
                    for _ in range(50):
                        ew += ba.expected_winnings(m.agent, 1000, mode='loss percentage')
                    results[ind, :] = tc.tensor((lr_list[i], gamma_list[j], tau_list[k], ep_rate[l], ew/50.))
                    ind += 1
                    
    return results

In [56]:
p = ba.BlackNet(out_mode='tanh')
m = ba.Model(p)
m.create_memory(20000)
m.set_params(128, 50000, 0.0005, 1., 0.75, (0.99, 0.25, 1/15000.))
m.train_agent()

In [57]:
av = 0
for _ in range(1000):
    av += ba.expected_winnings(m.agent, 100, mode = 'loss percentage')
print(av/1000)

2.083


In [69]:
def plot_value_function(agent):
    #prepare possible states for network evaluation, since we don't feed the actual card values directly
    card_values = (tc.arange(4., 21., dtype=tc.float32, requires_grad=False) - 4.)/27.
    dealer_values = (tc.arange(1., 10., dtype=tc.float32, requires_grad=False) - 1.)/9.
    
    #assemble values into states
    expanded_cv = card_values.unsqueeze(1).repeat(1, dealer_values.shape[0])
    expanded_dv = dealer_values.unsqueeze(0).repeat(card_values.shape[0], 1)
    combined = tc.stack((expanded_cv, expanded_dv), dim=-1).view(-1,2)
    
    with tc.no_grad():
        ace0 = tc.cat((combined[:, 0].unsqueeze(1), tc.zeros((combined.shape[0], 1)), combined[:, 1].unsqueeze(1)), dim=-1)
        ace1 = tc.cat((combined[:, 0].unsqueeze(1), tc.ones((combined.shape[0], 1)), combined[:, 1].unsqueeze(1)), dim=-1)
    
    #evaluate the agent's policy network on the states
    with tc.no_grad():
        ace0_policy_values = agent.policy_net(ace0).numpy()
        ace1_policy_values = agent.policy_net(ace1)
    
    #Y = card_values.numpy()
    #X = dealer_values.numpy()
    
    return ace0_policy_values

In [70]:
plot_value_function(m.agent)

array([[-8.78619969e-01, -1.62661061e-01],
       [-7.00762868e-01, -8.72896388e-02],
       [-6.66633606e-01, -7.12398812e-02],
       [-6.59242451e-01, -6.34769276e-02],
       [-6.48186862e-01, -6.01621456e-02],
       [-6.36451960e-01, -6.21176623e-02],
       [-6.33407652e-01, -7.05456883e-02],
       [-6.43681347e-01, -9.09421593e-02],
       [-6.66596055e-01, -1.28309771e-01],
       [-8.63453984e-01, -2.07459599e-01],
       [-6.71342850e-01, -1.22728840e-01],
       [-6.38712645e-01, -1.02603897e-01],
       [-6.32731974e-01, -9.15587023e-02],
       [-6.22764766e-01, -8.40979591e-02],
       [-6.11766100e-01, -8.05909038e-02],
       [-6.09802723e-01, -8.46367478e-02],
       [-6.22106910e-01, -1.04834102e-01],
       [-6.47956550e-01, -1.45370215e-01],
       [-8.45448732e-01, -2.45750964e-01],
       [-6.33826613e-01, -1.56240836e-01],
       [-6.01533830e-01, -1.35331765e-01],
       [-5.96926033e-01, -1.22280180e-01],
       [-5.88183343e-01, -1.09983750e-01],
       [-5.